<a href="https://colab.research.google.com/github/Art9521/Art9521/blob/main/Tg_wether_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import telebot
import requests
import json

bot = telebot.TeleBot('')
API = ''

# Словарь с переводом описаний погоды на русский язык
weather_descriptions = {
    "clear sky": "ясное небо",
    "few clouds": "небольшая облачность",
    "scattered clouds": "рассеянные облака",
    "broken clouds": "облачно с прояснениями",
    "shower rain": "ливень",
    "rain": "дождь",
    "light rain": "небольшой дождь",
    "thunderstorm": "гроза",
    "snow": "снег",
    "mist": "туман",
    "overcast clouds": "пасмурно"
}

def get_wind_direction(deg):
    if deg >= 337.5 or deg < 22.5:
        return "Cеверный"
    elif deg >= 22.5 and deg < 67.5:
        return "Cеверо-Восточный"
    elif deg >= 67.5 and deg < 112.5:
        return "Восточный"
    elif deg >= 112.5 and deg < 157.5:
        return "Юго-Восточный"
    elif deg >= 157.5 and deg < 202.5:
        return "Южный"
    elif deg >= 202.5 and deg < 247.5:
        return "Юго-Западный"
    elif deg >= 247.5 and deg < 292.5:
        return "Западный"
    elif deg >= 292.5 and deg < 337.5:
        return "Северо-Западный"

@bot.message_handler(commands=['start'])
def handle_start(message):
    name = message.from_user.first_name
    bot.send_message(message.chat.id, f"Приветствую тебя, {name}! Напиши название города, чтобы узнать погоду:")

@bot.message_handler(content_types=['text'])
def get_weather(message):
    city = message.text.strip().lower()
    res = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API}&units=metric')
    if res.status_code == 200:
        data = json.loads(res.text)
        country = data["sys"]["country"]
        temp = data["main"]["temp"]
        cloudiness = data["clouds"]["all"]
        wind_speed = data["wind"]["speed"]
        wind_direction = data["wind"]["deg"]
        pressure = data["main"]["pressure"]
        weather_id = data["weather"][0]["id"]
        weather_description = data["weather"][0]["description"]

        translated_description = weather_descriptions.get(weather_description, weather_description)

        # Добавление актуальных эмодзи для погоды
        if weather_id >= 200 and weather_id < 300:
            emoji = '⛈️'
        elif weather_id >= 300 and weather_id < 500:
            emoji = '🌧️'
        elif weather_id >= 500 and weather_id < 600:
            emoji = '🌦️'
        elif weather_id >= 600 and weather_id < 700:
            emoji = '❄️'
        elif weather_id >= 700 and weather_id < 800:
            emoji = '🌫️'
        elif weather_id == 800:
            emoji = '☀'
        elif weather_id > 800:
            emoji = '☁️'

        # Добавление советов в зависимости от погоды
        if temp < 0:
            advice = "На улице холодно. Одевайтесь теплее!"
            emojis = '🥶'
        elif temp < 10:
            advice = "Прохладно. Не забудьте надеть куртку."
            emojis = '🌀❄'
        elif temp < 20:
            advice = "Погода приятная. Подойдет легкая одежда."
            emojis = '🤗'
        else:
            advice = "Тепло. Одевайтесь по-летнему."
            emojis = '🌅'
        if cloudiness > 50:
            advice += " Также сегодня облачно, возможно потребуется зонт."
            emojis += '☔︎︎💧'
        if wind_speed > 10:
            advice += " Ветер сильный, будьте осторожны на улице."
            emojis += '🌪💨'

        wind_dir_text = get_wind_direction(wind_direction)

        # Форматированный вывод
        weather_message = (
            f'🗺 🌍 Локация: {country}\n'
            f'{emoji} Погода сейчас: {temp}°C\n'
            f'🌫 Облачность: {cloudiness}%\n'
            f'🌬️ Скорость ветра: {wind_speed} м/с\n'
            f'𓊹 Направление ветра: {wind_dir_text} {wind_direction}°\n'
            f'📉 Давление: {pressure} мм рт. ст.\n'
            f'🔍 На улице: {translated_description}\n\n'
            f'{advice} {emojis}\n\n'
            f'🌃 Можешь сразу присылать название нового города\n'
            f'⭐ Если что-то не работает отправь - /start\n'
        )

        bot.reply_to(message, weather_message)
    else:
        bot.reply_to(message, 'Неверное название города. Пожалуйста, проверьте введенное название.')

bot.polling(none_stop=True)
